# 数え上げ

## $nCk \pmod{p}$ の計算

https://drken1215.hatenablog.com/entry/2018/06/08/210000

### 方法1

$min(k, n-k) \leq 10^6$ 程度まで対応可能。

$$
\begin{align}
    _nC_k &= \frac{n!}{k!(n-k)!} = \frac{n(n-1) \ldots (n-k+1)}{k(k-1) \ldots 1} \\
    &= \left( \prod_{i=1}^{k} n-k+i \right) \left( \prod_{i=1}^{k} i \right)^{-1}
\end{align}
$$

分母、分子とも$k$個の整数の積なので、全体の計算量はクエリ毎に$O(k)$。

In [1]:
def nCk_mod_v1(n, k, p=10**9+7):
    k = min(k, n-k)
    a = 1
    for i in range(n-k+1, n+1):
        a = a * i % p
    b = 1
    for i in range(1, k+1):
        b = b * i % p
    return a * pow(b, p-2, p) % p

In [2]:
nCk_mod_v1(10, 4)

210

In [3]:
nCk_mod_v1(10**12, 10**6)

344082972

In [4]:
%timeit nCk_mod_v1(10**12, 10**6)

328 ms ± 55.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### 方法2

$n$が小さければ、方法1に比べてさらに高速化できる。  
$n \leq 10^6$ 程度まで対応可能。

$$_nC_k = \frac{n!}{k!(n-k)!} = n!(k!)^{-1}((n-k)!)^{-1}$$

$n$以下の全ての$k'$に対して$k'!, (k'!)^{-1}$ の2種類の値を事前計算しておくことで、クエリ毎に$O(1)$で計算可能となる。

$k'!$については逐次計算し、その値を記憶しておく。

$(k'!)^{-1}$の計算については、逆元を使って求めると計算量は$O(\log(p))$となる。  

In [5]:
def nCk_mod_v2_preprocess(n, p=10**9+7):
    f = [0 for i in range(n+1)]  # n!
    invf = [0 for i in range(n+1)]  # (n!)^-1
    f[0] = 1
    f[1] = 1
    invf[0] = 1
    invf[1] = 1
    for i in range(2, n+1):
        f[i] = f[i-1] * i % p
        invf[i] = pow(f[i], p-2, p)
    return f, invf

def nCk_mod_v2(it, p=10**9+7):
    nmax = max([n for n, k in it])
    f, invf = nCk_mod_v2_preprocess(nmax, p)
    for n, k in it:
        if n < k or n < 0 or k < 0:
            yield 0
        else:
            yield (f[n] * invf[k] % p) * invf[n-k] % p

前処理の `pow(f[i], p-2, p)` を含むループはさらに計算量を落とすことができる。  
$a_k = (k!)^{-1}$ とおけば、漸化式 $a_{k-1} = k \cdot a_k$ が成り立つので、 $a_n = (n!)^{-1}$ だけ計算すれば他は漸化式から求まる。

事前計算は$O(n)$、クエリ毎に$O(1)$。

In [6]:
def nCk_mod_v2_preprocess(n, p=10**9+7):
    f = [0 for i in range(n+1)]  # n!
    invf = [0 for i in range(n+1)]  # (n!)^-1
    f[0] = 1
    f[1] = 1
    invf[0] = 1
    invf[1] = 1
    for i in range(2, n+1):
        f[i] = f[i-1] * i % p
    invf[n] = pow(f[n], p-2, p)
    for i in range(n, 2, -1):
        invf[i-1] = invf[i] * i % p
    return f, invf

In [7]:
next(nCk_mod_v2([[10, 4], ]))

210

In [8]:
next(nCk_mod_v2([[10**6, 4], ]))

85121780

In [9]:
%timeit next(nCk_mod_v2([[10**6, 4], ]))

439 ms ± 8.36 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
